In [24]:
!pip install dwave-ocean-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 21.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.9/158.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.1/605.1 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 893.4/893.4 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.7/702.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.9/952.9 kB 23.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

## **Imports**

In [64]:
import numpy as np
import itertools
from dimod import BQM

from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from dimod.reference.samplers import ExactSolver


## **1. From Integer Linear Programming (ILP) to Quadratic Unconstrained Binary Optimization (QUBO)**

### **Define the ILP formulation of the BPP**

In [61]:

def BPP_ILP_Program(object_sizes, container_capacity):
    num_objects = len(object_sizes)  # Number of objects
    max_containers = num_objects  # Maximum number of containers (can have up to num_objects containers)

        # Create the quadratic program
    qp = QuadraticProgram()

    # Binary variables for containers
    container_usage_variables = {}
    for container in range(max_containers):
        container_var = f"y{container + 1}"  # Variable to indicate if the container is in use
        container_usage_variables[container_var] = 1  # Coefficient for minimization
        qp.binary_var(container_var)
    
    # Objective function: minimize the number of containers used
    qp.minimize(linear=container_usage_variables)

        # Constraint 1: Each object must be assigned to exactly one container
    for obj in range(num_objects):
        object_assignment_vars = {}
        for container in range(max_containers):
            assignment_var = f"x{obj+1}_{container+1}"  # Variable for object assignment to containers
            object_assignment_vars[assignment_var] = 1
            qp.binary_var(assignment_var)
        qp.linear_constraint(linear=object_assignment_vars, sense="==", rhs=1, name=f"x_{obj + 1}_assigned_const")

    # Constraint 2: The total size of objects in each container must not exceed its capacity
    for container in range(max_containers):
        container_load_vars = {}
        for obj in range(num_objects):
            assignment_var = f"x{obj+1}_{container+1}"
            container_load_vars[assignment_var] = object_sizes[obj]  # Size of the object
        
        container_var = f"y{container + 1}"
        container_load_vars[container_var] = -container_capacity  # Container capacity
        qp.linear_constraint(linear=container_load_vars, sense="<=", rhs=0, name=f"y_{container+1}_capacity_const")

    return qp



# Problem parameters
object_sizes = [4, 8, 1, 4]  # Sizes of the objects
container_capacity = 10  # Capacity of the containers

qp = BPP_ILP_Program(object_sizes, container_capacity)



Problem name: 

Minimize
  y1 + y2 + y3 + y4

Subject to
  Linear constraints (8)
    x1_1 + x1_2 + x1_3 + x1_4 == 1  'x_1_assigned_const'
    x2_1 + x2_2 + x2_3 + x2_4 == 1  'x_2_assigned_const'
    x3_1 + x3_2 + x3_3 + x3_4 == 1  'x_3_assigned_const'
    x4_1 + x4_2 + x4_3 + x4_4 == 1  'x_4_assigned_const'
    4*x1_1 + 8*x2_1 + x3_1 + 4*x4_1 - 10*y1 <= 0  'y_1_capacity_const'
    4*x1_2 + 8*x2_2 + x3_2 + 4*x4_2 - 10*y2 <= 0  'y_2_capacity_const'
    4*x1_3 + 8*x2_3 + x3_3 + 4*x4_3 - 10*y3 <= 0  'y_3_capacity_const'
    4*x1_4 + 8*x2_4 + x3_4 + 4*x4_4 - 10*y4 <= 0  'y_4_capacity_const'

  Binary variables (20)
    y1 y2 y3 y4 x1_1 x1_2 x1_3 x1_4 x2_1 x2_2 x2_3 x2_4 x3_1 x3_2 x3_3 x3_4 x4_1
    x4_2 x4_3 x4_4



### **Create a function to transform the ILP model into a QUBO**

In [65]:
def quadratic_program_to_qubo(qp):
    conv = QuadraticProgramToQubo()
    qubo_problem = conv.convert(qp)
    return qubo_problem

qubo_problem = quadratic_program_to_qubo(qp)

# Print the problem formulation
print(qubo_problem.prettyprint())

Problem name: 

Minimize
  85*x1_1^2 + 10*x1_1*x1_2 + 10*x1_1*x1_3 + 10*x1_1*x1_4 + 320*x1_1*x2_1
  + 40*x1_1*x3_1 + 160*x1_1*x4_1 + 40*x1_1*y_1_capacity_const@int_slack@0
  + 80*x1_1*y_1_capacity_const@int_slack@1
  + 160*x1_1*y_1_capacity_const@int_slack@2
  + 120*x1_1*y_1_capacity_const@int_slack@3 + 85*x1_2^2 + 10*x1_2*x1_3
  + 10*x1_2*x1_4 + 320*x1_2*x2_2 + 40*x1_2*x3_2 + 160*x1_2*x4_2
  + 40*x1_2*y_2_capacity_const@int_slack@0
  + 80*x1_2*y_2_capacity_const@int_slack@1
  + 160*x1_2*y_2_capacity_const@int_slack@2
  + 120*x1_2*y_2_capacity_const@int_slack@3 + 85*x1_3^2 + 10*x1_3*x1_4
  + 320*x1_3*x2_3 + 40*x1_3*x3_3 + 160*x1_3*x4_3
  + 40*x1_3*y_3_capacity_const@int_slack@0
  + 80*x1_3*y_3_capacity_const@int_slack@1
  + 160*x1_3*y_3_capacity_const@int_slack@2
  + 120*x1_3*y_3_capacity_const@int_slack@3 + 85*x1_4^2 + 320*x1_4*x2_4
  + 40*x1_4*x3_4 + 160*x1_4*x4_4 + 40*x1_4*y_4_capacity_const@int_slack@0
  + 80*x1_4*y_4_capacity_const@int_slack@1
  + 160*x1_4*y_4_capacity_const@int_s

## **Example BPP small**

In [60]:
sizes = [3, 2, 5]
bin_capacity = 6
model_ilp_small = BPP_ILP_Program(sizes, bin_capacity)
print(model_ilp_small.prettyprint())


Problem name: 

Minimize
  y_1 + y_2 + y_3

Subject to
  Linear constraints (6)
    x_1_1 + x_1_2 + x_1_3 == 1  'x_1_assigned_const'
    x_2_1 + x_2_2 + x_2_3 == 1  'x_2_assigned_const'
    x_3_1 + x_3_2 + x_3_3 == 1  'x_3_assigned_const'
    3*x_1_1 + 2*x_2_1 + 5*x_3_1 - 6*y_1 <= 0  'y_1_capacity_const'
    3*x_1_2 + 2*x_2_2 + 5*x_3_2 - 6*y_2 <= 0  'y_2_capacity_const'
    3*x_1_3 + 2*x_2_3 + 5*x_3_3 - 6*y_3 <= 0  'y_3_capacity_const'

  Binary variables (12)
    y_1 y_2 y_3 x_1_1 x_1_2 x_1_3 x_2_1 x_2_2 x_2_3 x_3_1 x_3_2 x_3_3



## **Example BPP medium**

In [62]:
sizes = [3, 2, 5, 4, 7, 6]
bin_capacity = 10

model_ilp_medium = BPP_ILP_Program(sizes, bin_capacity)
print(model_ilp_medium.prettyprint())


Problem name: 

Minimize
  y1 + y2 + y3 + y4 + y5 + y6

Subject to
  Linear constraints (12)
    x1_1 + x1_2 + x1_3 + x1_4 + x1_5 + x1_6 == 1  'x_1_assigned_const'
    x2_1 + x2_2 + x2_3 + x2_4 + x2_5 + x2_6 == 1  'x_2_assigned_const'
    x3_1 + x3_2 + x3_3 + x3_4 + x3_5 + x3_6 == 1  'x_3_assigned_const'
    x4_1 + x4_2 + x4_3 + x4_4 + x4_5 + x4_6 == 1  'x_4_assigned_const'
    x5_1 + x5_2 + x5_3 + x5_4 + x5_5 + x5_6 == 1  'x_5_assigned_const'
    x6_1 + x6_2 + x6_3 + x6_4 + x6_5 + x6_6 == 1  'x_6_assigned_const'
    3*x1_1 + 2*x2_1 + 5*x3_1 + 4*x4_1 + 7*x5_1 + 6*x6_1 - 10*y1
    <= 0  'y_1_capacity_const'
    3*x1_2 + 2*x2_2 + 5*x3_2 + 4*x4_2 + 7*x5_2 + 6*x6_2 - 10*y2
    <= 0  'y_2_capacity_const'
    3*x1_3 + 2*x2_3 + 5*x3_3 + 4*x4_3 + 7*x5_3 + 6*x6_3 - 10*y3
    <= 0  'y_3_capacity_const'
    3*x1_4 + 2*x2_4 + 5*x3_4 + 4*x4_4 + 7*x5_4 + 6*x6_4 - 10*y4
    <= 0  'y_4_capacity_const'
    3*x1_5 + 2*x2_5 + 5*x3_5 + 4*x4_5 + 7*x5_5 + 6*x6_5 - 10*y5
    <= 0  'y_5_capacity_const'
    3*

## **Example BPP large**

In [63]:
sizes = [3, 2, 5, 4, 7, 6, 1, 8, 9, 10]
bin_capacity = 15

model_ilp_large = BPP_ILP_Program(sizes, bin_capacity)
print(model_ilp_large.prettyprint())


Problem name: 

Minimize
  y1 + y10 + y2 + y3 + y4 + y5 + y6 + y7 + y8 + y9

Subject to
  Linear constraints (20)
    x1_1 + x1_10 + x1_2 + x1_3 + x1_4 + x1_5 + x1_6 + x1_7 + x1_8 + x1_9
    == 1  'x_1_assigned_const'
    x2_1 + x2_10 + x2_2 + x2_3 + x2_4 + x2_5 + x2_6 + x2_7 + x2_8 + x2_9
    == 1  'x_2_assigned_const'
    x3_1 + x3_10 + x3_2 + x3_3 + x3_4 + x3_5 + x3_6 + x3_7 + x3_8 + x3_9
    == 1  'x_3_assigned_const'
    x4_1 + x4_10 + x4_2 + x4_3 + x4_4 + x4_5 + x4_6 + x4_7 + x4_8 + x4_9
    == 1  'x_4_assigned_const'
    x5_1 + x5_10 + x5_2 + x5_3 + x5_4 + x5_5 + x5_6 + x5_7 + x5_8 + x5_9
    == 1  'x_5_assigned_const'
    x6_1 + x6_10 + x6_2 + x6_3 + x6_4 + x6_5 + x6_6 + x6_7 + x6_8 + x6_9
    == 1  'x_6_assigned_const'
    x7_1 + x7_10 + x7_2 + x7_3 + x7_4 + x7_5 + x7_6 + x7_7 + x7_8 + x7_9
    == 1  'x_7_assigned_const'
    x8_1 + x8_10 + x8_2 + x8_3 + x8_4 + x8_5 + x8_6 + x8_7 + x8_8 + x8_9
    == 1  'x_8_assigned_const'
    x9_1 + x9_10 + x9_2 + x9_3 + x9_4 + x9_5 + x9_6 + 

## **GG**

In [97]:
def prepareProblemDwave(object_sizes, container_capacity):
        
    num_objects = len(object_sizes)  # Number of objects
    max_containers = num_objects  # Maximum number of containers (can have up to num_objects containers)

    container_usage_variables = {}
    ys = []
    xs = []

    for container in range(max_containers):
        container_var = f"y{container + 1}"  # Variable to indicate if the container is in use
        container_usage_variables[container_var] = 1  # Coefficient for minimization

    
    bqm = BQM(container_usage_variables, {}, 0, "BINARY")

    for obj in range(num_objects):
        object_assignment_vars = {}
        for container in range(max_containers):
            assignment_var = f"x{obj+1}{container+1}"  # Variable for object assignment to containers
            object_assignment_vars[assignment_var] = 1
            bqm.add_linear_equality_constraint([object_assignment_vars], constant=1, lagrange_multiplier=1)

    # for i in range(0, max_containers):
    #     c1 = []
    #     for j in range(0, num_objects):
    #         strng = "x"+str(i+1)+str(j+1)
    #         xs.append(strng)
    #         c1.append((strng, object_sizes[j]))
    #     c1.append((ys[i], container_capacity))
    #     bqm.add_linear_inequality_constraint(c1, label="c1_"+str(i+1), lb = -container_capacity, ub=0, lagrange_multiplier=6)

    # for j in range(1, num_objects+1):
    #     c2 = []
    #     for x in xs:
    #         if x[2] == str(j):
    #             c2.append((x, 1))
    #     bqm.add_linear_equality_constraint(c2, constant=-1, lagrange_multiplier=6)

    return bqm


# Problem parameters
object_sizes = [4, 8, 1, 4]  # Sizes of the objects
container_capacity = 10  # Capacity of the containers

qp = prepareProblemDwave(object_sizes, container_capacity)
qp

ValueError: too many values to unpack (expected 2)

In [72]:
bqm_str = BQM(
    {"x1": -5.0, "x2": -3.0, "x3": -8.0, "x4": -6.0},
    {("x1", "x2"): 4, ("x1", "x3"): 8, ("x2", "x3"): 2, ("x3", "x4"): 10},
    0,
    "BINARY",
)

bqm_str

BinaryQuadraticModel({'x1': -5.0, 'x2': -3.0, 'x3': -8.0, 'x4': -6.0}, {('x2', 'x1'): 4.0, ('x3', 'x1'): 8.0, ('x3', 'x2'): 2.0, ('x4', 'x3'): 10.0}, 0.0, 'BINARY')

In [88]:


bqm = BQM({'y1': 1.0, 'y2': 1.0, 'y3': 1.0, 'y4': 1.0},
          {},
          0,
          "BINARY")
bqm

BinaryQuadraticModel({'y1': 1.0, 'y2': 1.0, 'y3': 1.0, 'y4': 1.0}, {}, 0.0, 'BINARY')